# Bio.tools | filling in missing publication information

This is a notebook for completing missing publication information in bio.tools using EuropePMC and saving open access publication information and corresponding tool name.

In [ ]:
from europe_pmc import EuropePMC
from bh24_literature_mining.utils import load_biotools_from_zip, save_to_json

# tools = load_biotools_from_json("../biotoolspub/biotools.json")
tools = load_biotools_from_zip("../biotoolspub/biotoolsdump.zip")

open_access_publications = []

pmc_client = EuropePMC()


for tool in tools:
    publication = tool['publication']

    pmc_publication = []

    for pub in publication:
        pub_id = pub.get('doi', 'pmid')
        
        if not pub_id:
            continue

        response = pmc_client.fetch(pub_id)

        if not response.data:
            continue
    
        pub = {key: pub.get(key) for key in ['doi', 'pmid', 'pmcid']}
        pmc_pub = {key: response.data.get(key) for key in ['doi', 'pmid', 'pmcid']}
       
        pmc_publication.append(pmc_pub if pub != pmc_pub else pub)

        if response.data.get('isOpenAccess') == 'Y':
            open_access_publications.append({tool['name'], pmc_pub})



    tool['publication'] = pmc_publication


save_to_json(tools, 'biotools_updated_publication.json')
save_to_json(open_access_publications, 'open_access_publications.json')
